# Optimization Notebook

In [131]:
# modules
import numpy as np
import scipy.io as io
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [137]:
def newton(f,Df,x0,epsilon,max_iter=1000):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> Df = lambda x: 2*x - 1
    >>> newton(f,Df,1,1e-8,10)
    Found solution after 5 iterations.
    1.618033988749989
    '''
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

def newton_mat(f,Df,DDf,x0,epsilon,max_iter=1000,lam=1):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative (gradient) of f(x).
    Df : function
        Second derivative (Hessian) of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.
    lam : integer
        An extra parameter lambda which scales the correction term

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - Df(xn)/DDf(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.
    -------
    '''
    
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        DDfxn = DDf(xn)
        try:
            xn = xn - lam*np.dot(la.inv(DDfxn),np.transpose(Df(xn)))
        except np.linalg.linalg.LinAlgError as err:
            print("Hessian not invertible!")
            return err
    print('Exceeded maximum iterations. No solution found.')
    return None

In [138]:
def v(x: np.ndarray,A: np.ndarray):
    return np.ones(np.shape(A)[0]) - np.dot(A,x) 

def Phi(x: np.ndarray,A: np.ndarray):
    u = v(x,A)
    return np.dot(u,u)

def DPhi(x: np.ndarray,A: np.ndarray):
    ones = np.ones(np.shape(A)[0])
    print(ones)
    return  -2*np.dot(ones,A) + 2*np.dot(np.transpose(A),np.dot(A,x))

def DDPhi(x: np.ndarray, A: np.ndarray):
    return 2*np.dot(np.transpose(A),A)

In [139]:
A = np.array([[0,-1,2],
              [3,4,5]])
x = np.array([-2,3,4])
print(np.shape(x))
print(np.dot(A,np.ones(np.shape(A)[1])))
print(DPhi(x,A))
print(DDPhi(x,A))
la.inv(DDPhi(x,A))

(3,)
[ 1. 12.]
[1. 1.]
[150. 192. 266.]
[[18 24 30]
 [24 34 36]
 [30 36 58]]


array([[-4.06574966e+14,  1.87649984e+14,  9.38249922e+13],
       [ 1.87649984e+14, -8.66076851e+13, -4.33038426e+13],
       [ 9.38249922e+13, -4.33038426e+13, -2.16519213e+13]])

## Implement Newton Iteration for Matrices

In [150]:
A = io.mmread("/home/isaacmartin/Desktop/constrained-nonlinear-optimization/data/matrices/MUL_2x2x1/Cmat.mm").toarray()
x = np.ones(A.shape[1])

def v(x: np.ndarray):
    return np.ones(np.shape(A)[0]) - np.dot(A,x) 

def Phi(x: np.ndarray):
    u = np.ones(np.shape(A)[0]) - A.dot(x)
    return np.dot(u,u)

def DPhi(x: np.ndarray):
    ones = np.ones(np.shape(A)[0])
    print(ones.shape, A.shape)
    return  -2*ones.dot(A) + 2*np.transpose(A).dot(A.dot(x))

def DDPhi(x: np.ndarray):
    print(A)
    print(np.dot(np.transpose(A),A))
    return 2*np.transpose(A).dot(A)

newton_mat(f = Phi, Df = DPhi, DDf = DDPhi, x0 = x, epsilon = 0.1)

(480,) (480, 35)
[[ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  1. ... -1.  0.  0.]
 ...
 [ 0.  1.  1. ...  0.  0.  0.]
 [ 0.  1.  1. ...  1.  0.  0.]
 [ 0.  1.  1. ...  1.  0.  0.]]
[[256.  64.  16. ... -48.   0.   0.]
 [ 64. 256.  48. ... -48.   0.   0.]
 [ 16.  48. 256. ... -64.   0.   0.]
 ...
 [-48. -48. -64. ... 256.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
Hessian not invertible!


numpy.linalg.LinAlgError('singular matrix')